<a href="https://colab.research.google.com/github/TheNewLearn/bigdata_hw2/blob/main/bigdata_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from google.colab import drive
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
spark = SparkSession.builder.master("local").getOrCreate()
sc = SparkContext.getOrCreate()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jre-headless is already the newest version (8u292-b10-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
drive.mount("/content/drive")
path = "/content/drive/My Drive/News_Final.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
sqlContext = SQLContext(sc)
dataset = sqlContext.read.format("csv").options(header=True, inferSchema='True', quotes="\"",escape="\"").load(path);
dataset.show()



/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-------+--------------------+--------------------+--------------------+---------+-------------------+-------------------+--------------------+--------+----------+--------+
| IDLink|               Title|            Headline|              Source|    Topic|        PublishDate|     SentimentTitle|   SentimentHeadline|Facebook|GooglePlus|LinkedIn|
+-------+--------------------+--------------------+--------------------+---------+-------------------+-------------------+--------------------+--------+----------+--------+
|99248.0|Obama Lays Wreath...|Obama Lays Wreath...|           USA TODAY|    obama|2002-04-02 00:00:00|                0.0| -0.0533001790889026|      -1|        -1|      -1|
|10423.0|A Look at the Hea...|Tim Haywood, inve...|           Bloomberg|  economy|2008-09-20 00:00:00|  0.208333333333333|  -0.156385810542806|      -1|        -1|      -1|
|18828.0|Nouriel Roubini: ...|Nouriel Roubini, ...|           Bloomberg|  economy|2012-01-28 00:00:00| -0.425210032135381|   0.13975424

In [ ]:
#Total news title word freq
news_title = dataset.rdd.map(lambda x:x[1]).flatMap(lambda x:x.split(" ")).map(lambda line: (line,1)).reduceByKey(lambda a,b:a+b)
sorted(news_title.collect(),key = lambda x: x[1],reverse=True)
#Total news headline word freq
news_handline = dataset.rdd.filter(lambda x: type(x[2]) != type(None) ).map(lambda x:x[2]).flatMap(lambda x:x.split(" ")).map(lambda line: (line,1)).reduceByKey(lambda a,b:a+b)
sorted(news_handline.collect(),key = lambda x: x[1],reverse=True)

#news title word freq per day
news_perday_title = dataset.rdd.flatMap(lambda x : [((x[5][0:10],w),1) for w in x[1].split(" ")]).reduceByKey(lambda x,y: x+y)
sorted(news_perday_title.collect(),key = lambda x: x[1],reverse=True)
#news headline word freq per day
news_perday_head = dataset.rdd.filter(lambda x: type(x[2]) != type(None) ).flatMap(lambda x : [((x[5][0:10],w),1) for w in x[2].split(" ")]).reduceByKey(lambda x,y: x+y)
sorted(news_perday_head.collect(),key = lambda x: x[1],reverse=True)

#news title word freq per topic
news_pertopic_title = dataset.rdd.flatMap(lambda x : [((x[4],w),1) for w in x[1].split(" ")]).reduceByKey(lambda x,y: x+y)
sorted(news_pertopic_title.collect(),key = lambda x: x[1],reverse=True)

#news headline word freq per topic
news_pertopic__head = dataset.rdd.filter(lambda x: type(x[2]) != type(None) ).flatMap(lambda x : [((x[4],w),1) for w in x[2].split(" ")]).reduceByKey(lambda x,y: x+y)
sorted(news_pertopic__head.collect(),key = lambda x: x[1],reverse=True)

  



[(('economy', 'the'), 56176),
 (('obama', 'the'), 38402),
 (('microsoft', 'the'), 25773),
 (('obama', 'Obama'), 24236),
 (('obama', 'to'), 24192),
 (('economy', 'of'), 23291),
 (('economy', 'to'), 22168),
 (('economy', 'in'), 21151),
 (('economy', 'a'), 19406),
 (('microsoft', 'Microsoft'), 19398),
 (('obama', 'President'), 18985),
 (('economy', 'economy'), 18280),
 (('economy', 'and'), 17532),
 (('obama', 'a'), 17378),
 (('obama', 'of'), 17285),
 (('microsoft', 'to'), 17191),
 (('obama', 'in'), 14667),
 (('microsoft', 'a'), 14357),
 (('microsoft', 'of'), 13550),
 (('obama', 'Barack'), 12993),
 (('obama', 'and'), 12898),
 (('microsoft', 'and'), 12301),
 (('palestine', 'the'), 12180),
 (('economy', 'is'), 10827),
 (('obama', 'on'), 10823),
 (('economy', 'The'), 9920),
 (('economy', 'on'), 8783),
 (('microsoft', 'in'), 8751),
 (('microsoft', 'for'), 7791),
 (('economy', 'that'), 7752),
 (('economy', 'for'), 7653),
 (('microsoft', 'is'), 7480),
 (('obama', 'that'), 7454),
 (('microsoft', 

In [ ]:
from pyspark.sql.functions import col, lit

def bydaycsv(data,status):
  dataframe = data
  cols = ["IDLink","mean_day_1","mean_day_2"]
  if status == 0:
    dataframe.select(*cols).coalesce(1).write.format('com.databricks.spark.csv').save('Facebook_mean_perday.csv',header = 'true')
  elif status == 1:
    dataframe.select(*cols).coalesce(1).write.format('com.databricks.spark.csv').save('Google_mean_perday.csv',header = 'true')
  elif status == 2:
    dataframe.select(*cols).coalesce(1).write.format('com.databricks.spark.csv').save('Linkedln_mean_perday.csv',header = 'true')

def byhourcsv(data,status):
  cols = ["IDLink"]
  for i in range(0,48):
    cols.append("mean_hour_"+str(i+1))
  dataframe = data
  if status == 0:
    dataframe.select(*cols).coalesce(1).write.format('com.databricks.spark.csv').save('Facebook_mean_hour.csv',header = 'true')
  elif status == 1:
    dataframe.select(*cols).coalesce(1).write.format('com.databricks.spark.csv').save('Google_mean_hour.csv',header = 'true')
  elif status == 2:
    dataframe.select(*cols).coalesce(1).write.format('com.databricks.spark.csv').save('/content/drive/My Drive/inkedln_mean_hour.csv',header = 'true')







dataset = sqlContext.read.format("csv").options(header=True, inferSchema='True', quotes="\"",escape="\"").load(path)
#Facebook
Facebook_pathlist = ["/content/drive/My Drive/Facebook/Facebook_Microsoft.csv","/content/drive/My Drive/Facebook/Facebook_Obama.csv","/content/drive/My Drive/Facebook/Facebook_Palestine.csv"]
fe_dc = sqlContext.read.format("csv").options(header=True, inferSchema='True', quotes="\"",escape="\"").load("/content/drive/My Drive/Facebook/Facebook_Economy.csv")
for i in Facebook_pathlist:
  fe_df = sqlContext.read.format("csv").options(header=True, inferSchema='True', quotes="\"",escape="\"").load(i)
  fe_dc = fe_dc.union(fe_df)


rowMean  = (sum(col(x) for x in fe_dc.columns[1:73]) / 72).alias("mean")
fb_perday = fe_dc.withColumn('mean_day_1',rowMean)
rowMean  = (sum(col(x) for x in fe_dc.columns[73:145]) / 72).alias("mean")
fb_perday = fb_perday.withColumn('mean_day_2',rowMean)


rowMean  = (sum(col(x) for x in fe_dc.columns[1:4]) / 3).alias("mean")
fb_perhour = fe_dc.withColumn('mean_hour_1',rowMean)

timecount = 2
for i in range(4,145,3):
  rowMean  = (sum(col(x) for x in fe_dc.columns[i:i+3]) / 3).alias("mean")
  fb_perhour = fb_perhour.withColumn('mean_hour_'+str(timecount),rowMean)
  timecount+=1







#Google
Google_pathlist = ["/content/drive/My Drive/Google/GooglePlus_Microsoft.csv","/content/drive/My Drive/Google/GooglePlus_Obama.csv","/content/drive/My Drive/Google/GooglePlus_Palestine.csv"]
ge_dc = sqlContext.read.format("csv").options(header=True, inferSchema='True', quotes="\"",escape="\"").load("/content/drive/My Drive/Google/GooglePlus_Economy.csv")
for i in Google_pathlist:
  ge_df = sqlContext.read.format("csv").options(header=True, inferSchema='True', quotes="\"",escape="\"").load(i)
  ge_dc = ge_dc.union(ge_df)





rowMean  = (sum(col(x) for x in ge_dc.columns[1:73]) / 72).alias("mean")
google_perday = ge_dc.withColumn('mean_day_1',rowMean)
rowMean  = (sum(col(x) for x in ge_dc.columns[73:145]) / 72).alias("mean")
google_perday = google_perday.withColumn('mean_day_2',rowMean)
#google_perday.show()

rowMean  = (sum(col(x) for x in ge_dc.columns[1:4]) / 3).alias("mean")
google_perhour = ge_dc.withColumn('mean_hour_1',rowMean)

gtimecount = 2
for i in range(4,145,3):
  rowMean  = (sum(col(x) for x in ge_dc.columns[i:i+3]) / 3).alias("mean")
  google_perhour = google_perhour.withColumn('mean_hour_'+str(gtimecount),rowMean)
  gtimecount+=1

#Linkedln

Linkedln_pathlist = ["/content/drive/My Drive/Linkedln/LinkedIn_Microsoft.csv","/content/drive/My Drive/Linkedln/LinkedIn_Obama.csv","/content/drive/My Drive/Linkedln/LinkedIn_Palestine.csv"]
lk_dc = sqlContext.read.format("csv").options(header=True, inferSchema='True', quotes="\"",escape="\"").load("/content/drive/My Drive/Linkedln/LinkedIn_Economy.csv")
for i in Linkedln_pathlist:
  lk_df = sqlContext.read.format("csv").options(header=True, inferSchema='True', quotes="\"",escape="\"").load(i)
  lk_dc = lk_dc.union(lk_df)




rowMean  = (sum(col(x) for x in lk_dc.columns[1:73]) / 72).alias("mean")
Linkedln_perday = lk_dc.withColumn('mean_day_1',rowMean)
rowMean  = (sum(col(x) for x in lk_dc.columns[73:145]) / 72).alias("mean")
Linkedln_perday = Linkedln_perday.withColumn('mean_day_2',rowMean)
#Linkedln_perday.show()

rowMean  = (sum(col(x) for x in lk_dc.columns[1:4]) / 3).alias("mean")
Linkedln_perhour = lk_dc.withColumn('mean_hour_1',rowMean)
lktimecount = 2
for i in range(4,145,3):
  rowMean  = (sum(col(x) for x in lk_dc.columns[i:i+3]) / 3).alias("mean")
  Linkedln_perhour = Linkedln_perhour.withColumn('mean_hour_'+str(lktimecount),rowMean)
  lktimecount+=1

byhourcsv(Linkedln_perhour,2)
#Linkedln_perhour.show()



  







  




Py4JError: ignored

In [ ]:
def sum_sentiment(dataset,topic,status):
  if status == 0:
    df = dataset.rdd.filter(lambda x:x[4] == topic)
    result = df.map(lambda x:x[6]).reduce(lambda x,y: x+y)
    mean = df.map(lambda x:x[6]).map(lambda x: float(x))
    mean = mean.mean()
    return result, mean
  elif status == 1:
    df = dataset.rdd.filter(lambda x:x[4] == topic)
    result = df.map(lambda x:x[7]).reduce(lambda x,y: x+y)
    mean = df.map(lambda x:x[7]).map(lambda x: float(x))
    mean = mean.mean()
    return result, mean





#obama Sentiment
obama_st = sum_sentiment(dataset,"obama",0)
obama_sh = sum_sentiment(dataset,"obama",1)

#Economy Sentiment
economy_st = sum_sentiment(dataset,"economy",0)
economy_sh = sum_sentiment(dataset,"economy",1)

#microsoft Sentiment
microsoft_st = sum_sentiment(dataset,"microsoft",0)
microsoft_sh = sum_sentiment(dataset,"microsoft",1)

#palestine Sentiment
palestine_st = sum_sentiment(dataset,"palestine",0)
palestine_sh = sum_sentiment(dataset,"palestine",1)

print(obama_st)
print(obama_sh)


print(economy_st)
print(economy_sh)

print(palestine_st)
print(palestine_sh)


print(microsoft_st)
print(microsoft_sh)



(-27.604435365771188, -0.0009648526866749828)
(-507.62019742134794, -0.017742754191588533)
(-350.89723953455365, -0.01034240861632135)
(-1340.1388172155725, -0.03949949355150792)
(-177.61514431068505, -0.020085394584494515)
(-393.38238008336225, -0.04448517246221418)
(51.56650733416493, 0.0023591594534799728)
(-322.2826752054217, -0.014744380785315318)


In [ ]:
#task 4

def toplist(dataset,topic):
  lt = dataset.filter(lambda x:x[0][0] == topic)
  lt = sorted(lt.collect(),key= lambda x:x[1],reverse=True)
  lt_top = []
  for i in range(0,100):
    lt_top.append(lt[i][0][1])
  return lt_top

def co_matrix(toplist,allword):
  co_matrix = []
  for a in range(0,len(toplist)):
    co_matrix.append([])
    for b in range(0,len(toplist)):
      count = 0
      for x in allword:
        if toplist[a] in x and toplist[b] in x:
          count+=1
      co_matrix[a].append(count)
  return co_matrix


all_new_title = dataset.rdd.map(lambda x:(x[1])).collect()
all_new_headline = dataset.rdd.filter(lambda x: type(x[2]) != type(None)).map(lambda x:(x[2])).collect()
topic_list = ['obama','economy','palestine','microsoft']
for i in topic_list:
  title_toplist = toplist(news_pertopic_title,i)
  headline_toplist = toplist(news_pertopic__head,i)
  print(co_matrix(title_toplist,all_new_title))
  print(co_matrix(headline_toplist,all_new_headline))






    #print(str(a)+" "+str(b))
#print(toplist(news_pertopic_title,"obama"))




[[27919, 10674, 7388, 5041, 13188], [10674, 31567, 7929, 1513, 19367], [7388, 7929, 22740, 1664, 13152], [5041, 1513, 1664, 5041, 2394], [13188, 19367, 13152, 2394, 56683]]
[[74471, 21387, 50146, 15675, 74459], [21387, 27227, 19341, 18576, 27227], [50146, 19341, 61914, 14321, 61894], [15675, 18576, 14321, 20005, 20004], [74459, 27227, 61894, 20004, 93169]]
[[18095, 6122, 69, 2725, 3566], [6122, 31567, 2126, 7929, 4345], [69, 2126, 8528, 1349, 1824], [2725, 7929, 1349, 22740, 4016], [3566, 4345, 1824, 4016, 14019]]
[[74471, 48769, 50146, 68401, 74459], [48769, 58988, 39256, 53827, 58962], [50146, 39256, 61914, 56942, 61894], [68401, 53827, 56942, 84892, 84864], [74459, 58962, 61894, 84864, 93169]]
[[4505, 1392, 4505, 811, 83], [1392, 31567, 19503, 7929, 570], [4505, 19503, 55821, 14048, 2031], [811, 7929, 14048, 22740, 506], [83, 570, 2031, 506, 2031]]
[[74471, 48769, 50146, 74459, 68401], [48769, 58988, 39256, 58962, 53827], [50146, 39256, 61914, 61894, 56942], [74459, 58962, 61894, 93